<a href="https://colab.research.google.com/github/Reclone-org/DNA-Scripts/blob/main/RecloneSyntax_PrepareCDSforSynthesis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Preparing a protein sequence for assembly with the Reclone Syntax

# 🧬 Notebook purpose

This notebook takes coding sequences (preferably **protein sequences**) and produces **synthesis-ready DNA parts** for Reclone-style assembly:

* Optimizes codons for *E. coli* (DNA Chisel).
* Enriches wobble bases in **codons 1–7** (relative to ATG) to favor A (and T for the first codon’s wobble).
* Minimizes guanosines (G) via synonymous changes where possible (within codons 1–7).
* Checks/avoids common restriction sites, long homopolymers, and enforces GC% windows (DNA Chisel).
* Builds adaptered parts with **internal BsaI tags** (your chosen 5′/3′ syntax letters) and outer **BbsI** sites/overhangs for direct cloning of fragments into pOpenv3 (left `CGCT`, right `GGAG`) plus spacers. NB: we are currently [deliberating on storage vectors for Reclone](https://forum.reclone.org/t/reclone-storage-vectors-future-directions/1190/4) and may deprectae pOpen_v3.
* Verifies adapter layout with **guardrails** so tags land on the **correct ends**.

You’ll get two FASTAs:

1. **Enriched inserts** (no adapters)
2. **Synthesis-ready parts** (with BsaI tags + spacers + BbsI adapters)

(Optionally, a protein translation FASTA of the enriched inserts, if enabled.)

---

# 📄 Input CSV format

When prompted, upload a CSV with these columns (case-insensitive headers are accepted):

* **Name** — your part name (e.g., `mVirD2`, `sfGFP`).
* **Seq** — the **protein sequence** (single-letter AAs).

  * If you include a leading **M**, that’s fine; the code treats the insert as starting **after** the start codon (ATG).
  * If you accidentally provide **DNA** (only A/C/G/T), the notebook will print a warning and can still proceed.
* **5' syntax position** — the **5′ tag** (left tag).
* **3' syntax position** — the **3′ tag** (right tag).
* **Enzyme** — optional; defaults to **BsaI** for internal tags. (Other enzymes are allowed but aren’t auto-handled by the Reclone map.)

### Allowed syntax tags

A, B, C, D, E, F, N1, N2, N3, N4, N5


> **Important:** The first tag (5′ column) is applied to the **left** end of the insert; the second tag (3′ column) to the **right** end.
> Example: a row with `C` (5′) and `D` (3′) builds a **CD** part.

### Example CSV

```
Name,Seq,5' syntax position,3' syntax position,Enzyme
mVirD2,SEQPTRWQ...K,C,D,BsaI
sfGFP,MSKGEELFTGVVP...,B,C,BsaI
tagRFP,TGSHHHHHHGSG, N1, N5, BsaI
```

---

# 🔧 What the notebook does (pipeline)

1. **Load CSV** and validate tags.

   * Warns if `Seq` looks like plain DNA (only A/C/G/T).
2. **(If Seq is protein)** Reverse-translate to DNA (bacterial table 11), favoring **low-G codons**.
3. **DNA Chisel optimization:**

   * `CodonOptimize(species="e_coli")`
   * Avoids: BsaI/SapI/BbsI/BsmBI/EcoRI/NotI/XbaI/SpeI/PstI/BtgZI/AarI/NheI/XhoI/BamHI/BglII/NruI
   * Avoids homopolymers ≥6 (A/T/C/G)
   * Enforces GC% in sliding windows (default 40–60% in 100 bp)
4. **Early-codon enrichment (codons 1–7, after ATG):**

   * Codon 1 wobble ∈ {A,T}; codons 2–7 wobble = A.
   * Then reduce G by synonymous changes (still within codons 1–7).
   * Every edit preserves the amino-acid sequence and re-checks constraints.
5. **Adapter assembly:**

   * Internal **BsaI** tags from your 5′/3′ letters (position-aware scars as defined), plus linkers/spacers.
   * Outer **BbsI** sites with fixed pOpenv3 overhangs: left `CGCT`, right `GGAG`.
6. **Guardrails:**

   * Verifies exact positions of BbsI sites/overhangs, BsaI sites, and your tag cores + scar bases on both ends.
7. **Insert-only constraint check** inside the assembled construct (adapters are ignored in this check to avoid false failures).
8. **Outputs** are written and a per-record **log** is printed with GC%, #edits, ΔG, and guardrail status.

---

# 📦 Outputs you’ll see

* `optimized_sequences_enriched_checked.fasta`
  Enriched, constraint-checked **inserts only** (no adapters).
  Description includes number of edits, AA/constraint status.

* `synthesis_parts.fasta`
  **Synthesis-ready parts** with your chosen 5′/3′ BsaI tags, spacers, and outer BbsI adapters.
  IDs include the part name and the tag pair (e.g., `|CD`).

* *(Optional)* `optimized_inserts_translation.fasta`
  Protein translations of `ATG + insert` (helpful sanity check).

---

# ⚠️ Common messages & what they mean

* **“Seq contains only A/C/G/T (looks like DNA)”**
  Your `Seq` appears to be DNA, not protein. If intentional, you can proceed; otherwise replace with the protein sequence.

* **Guardrail FAIL**
  A tag/site didn’t land where expected. The log shows which token failed (e.g., `Right_Bsa_core`). Check your tags and spacers.

* **Constraint FAIL (after edits)**
  DNA Chisel found a forbidden motif or GC/homopolymer issue after enrichment. The summary tells you where; adjust the input or relax constraints.

---

# ▶️ Quick start

1. Prepare a CSV like the example above.
2. Run the notebook cells in order.
3. Watch the per-row logs for PASS/FAIL statuses.
4. Download:

   * **Inserts** → `optimized_sequences_enriched_checked.fasta`
   * **Parts**   → `synthesis_parts.fasta`
   * (Optional) **Protein translations** → `optimized_inserts_translation.fasta`

---

# 📝 Notes

* The insert is assumed to start **immediately after** the start codon (**ATG**). If your protein `Seq` begins with **M**, that’s fine—the pipeline accounts for it.
* Overhang definitions are **position-aware**: the first tag uses its **5′** spec; the second tag uses its **3′** spec (including any scar bases).
* Outer BbsI overhangs for pOpenv3 are fixed: **left `CGCT`**, **right `GGAG`**.

If you want different scar defaults (e.g., serine instead of glycine for specific letters), edit the **5′/3′ overhang map** near the top of the notebook.


## Install packages

In [ ]:
!apt-get remove -y swig
!apt-get install -y swig3.0
!ln /usr/bin/swig3.0 /usr/bin/swig

# Install necessary packages with verbose output
!apt-get update
!apt-get install -y build-essential libglpk-dev zlib1g-dev
# Added --upgrade --force-reinstall for biopython
!pip install biopython dnachisel pydna matplotlib Bio


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
Package 'swig' is not installed, so not removed
0 upgraded, 0 newly installed, 0 to remove and 50 not upgraded.
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
swig3.0 is already the newest version (3.0.12-2.2ubuntu1).
0 upgraded, 0 newly installed, 0 to remove and 50 not upgraded.
ln: failed to create hard link '/usr/bin/swig': File exists
Hit:1 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 https://cli.github.com/packages stable InRelease
Hit:4 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:7 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:8 http://archive.ubuntu.com/ubuntu jammy-ba

## Import relevant libraries

In [ ]:
# ========================= Imports =========================
from typing import List, Tuple, Dict, Optional, Set
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from Bio import SeqIO
from Bio.Data import CodonTable
from dnachisel import (
    DnaOptimizationProblem, AvoidPattern, EnforceGCContent, EnforceTranslation,
    CodonOptimize, HomopolymerPattern
)


## Set up adapter sites for Type IIS Assembly




In [ ]:

# ========================= Adapter constants (fixed for pOpenv3) =========================
BBSI_SITE_FW = "GAAGAC"
BBSI_SITE_RV = "GTCTTC"
BBSI_LEFT_OVERHANG  = "CGCT"
BBSI_RIGHT_OVERHANG = "GGAG"
BSAI_SITE_FW = "GGTCTC"
BSAI_SITE_RV = "GAGACC"

DEFAULT_FLANK5 = "tgatc"
DEFAULT_FLANK3 = "tacgt"
DEFAULT_BBSI_NN = "at"
DEFAULT_BSAI_N  = "a"
DEFAULT_SPACER_BETWEEN_SITES = "atgct"
DEFAULT_SPACER_RIGHT         = "tgcat"


# ========================= Reclone overhangs (position-aware) =========================
# Canonical 4-nt overhang for each letter
RECLONE_CANON: Dict[str, str] = {
    "A":  "GGAG",
    "B":  "TACT",
    "N1": "CCAT",
    "N2": "GTCA",
    "N3": "TCCA",
    "C":  "AATG",
    "D":  "AGGT",
    "N4": "TTCG",
    "N5": "CGGC",
    "E":  "GCTT",
    "F":  "CGCT",
    # (Optional back-compat) generic N alias; remove if not used:
    "N":  "TCCA",
}

# Position-specific specs (5′ and 3′) including any extra scar bases.
# Lowercase = extra bases outside the canonical 4-nt overhang.
# List reflects your corrected table (defaults to gly/met where shown).
RECLONE_END_SPECS: Dict[str, Dict[str, str]] = {
    "A":  {"5": "GGAG",    "3": "GGAG"},
    "B":  {"5": "TACT",    "3": "TACT"},
    "N1": {"5": "CCATg",   "3": "tCCAT"},
    "N2": {"5": "GTCA",    "3": "ggGTCA"},
    "N3": {"5": "TCCAtg",  "3": "TCCA"},
    "C":  {"5": "AATG",    "3": "ggAATG"},
    "D":  {"5": "AGGT",    "3": "ggAGGT"},
    "N4": {"5": "TTCG",  "3": "ggTTCG"},
    "N5": {"5": "CGGC",    "3": "ggCGGC"},
    "E":  {"5": "GCTT",    "3": "GCTT"},
    "F":  {"5": "CGCT",    "3": "CGCT"},
    # (Optional back-compat) generic N uses plain cores both ends:
    "N":  {"5": "TCCA",    "3": "TCCA"},
}

def _parse_end_spec(spec: str, canonical: str) -> Tuple[str, str, str]:
    """
    Parse a position-specific spec into (prefix, overhang, suffix).
    - 'spec' may include lowercase extras and spaces (e.g., "ggG TCA" -> prefix="gg", overhang="GTCA", suffix="").
    - 'canonical' must match the uppercase core extracted from spec.
    Returns uppercase strings.
    """
    s = spec.replace(" ", "")
    # leading lowercase prefix
    i = 0
    while i < len(s) and s[i].islower():
        i += 1
    # trailing lowercase suffix
    j = len(s) - 1
    while j >= 0 and s[j].islower():
        j -= 1
    prefix = s[:i]
    core   = s[i:j+1] if j >= i else ""
    suffix = s[j+1:] if j+1 < len(s) else ""
    if core.upper() != canonical:
        raise ValueError(f"Spec '{spec}' core '{core.upper()}' != canonical overhang '{canonical}'")
    return prefix.upper(), canonical, suffix.upper()

def resolve_bsa_end(letter: str, side: str) -> Tuple[str, str, str]:
    """
    Resolve a BsaI overhang 'letter' at 5′ or 3′ into (prefix, overhang, suffix), all uppercase.
    'side' must be '5' or '3'.
    """
    L = letter.strip().upper()
    if L not in RECLONE_CANON:
        raise ValueError(f"Unknown overhang letter '{letter}'. Valid: {', '.join(sorted(RECLONE_CANON))}")
    if L not in RECLONE_END_SPECS or side not in RECLONE_END_SPECS[L]:
        raise ValueError(f"No end spec for {L} at {side}′.")
    return _parse_end_spec(RECLONE_END_SPECS[L][side], RECLONE_CANON[L])

## Set up domesticatation and codon optimisation constraints for E.coli

In [ ]:
# ========================= Global config =========================
def build_constraints():
    return [
        AvoidPattern("BsaI_site", strand='both'),
        AvoidPattern("SapI_site", strand='both'),
        AvoidPattern("BbsI_site", strand='both'),
        AvoidPattern("BsmBI_site", strand='both'),
        AvoidPattern("EcoRI_site", strand='both'),
        AvoidPattern("NotI_site", strand='both'),
        AvoidPattern("XbaI_site", strand='both'),
        AvoidPattern("SpeI_site", strand='both'),
        AvoidPattern("PstI_site", strand='both'),
        AvoidPattern("BtgZI_site", strand='both'),
        AvoidPattern("AarI_site", strand='both'),
        AvoidPattern("NheI_site", strand='both'),
        AvoidPattern("XhoI_site", strand='both'),
        AvoidPattern("BamHI_site", strand='both'),
        AvoidPattern("BglII_site", strand='both'),
        AvoidPattern("NruI_site", strand='both'),
        AvoidPattern(HomopolymerPattern("A", 6)),
        AvoidPattern(HomopolymerPattern("T", 6)),
        AvoidPattern(HomopolymerPattern("C", 6)),
        AvoidPattern(HomopolymerPattern("G", 6)),
        EnforceGCContent(mini=0.4, maxi=0.6, window=100),
        EnforceTranslation(genetic_table='Bacterial'),
    ]


## Step 1: Import a list of protein sequences of interest

In [ ]:
# --- CSV loader (two separate tag columns) ---
import csv
from typing import List, Dict, Any
from collections import Counter
from google.colab import files

# Allowed tag tokens (5′/3′ syntax letters)
VALID_TAGS = {"A","B","C","D","E","F","N1","N2","N3","N4","N5","N"}  # 'N' optional legacy

def _ci_lookup(row: Dict[str, Any], *candidates: str) -> str | None:
    """Case-insensitive lookup of a field name from a CSV DictReader row."""
    lower_map = {k.lower(): k for k in row.keys()}
    for c in candidates:
        k = lower_map.get(c.lower())
        if k is not None:
            val = row.get(k)
            if val is not None and str(val).strip() != "":
                return str(val).strip()
    return None

def _looks_like_plain_dna(seq: str) -> bool:
    s = (seq or "").upper().replace(" ", "")
    return len(s) > 0 and set(s) <= set("ACGT")

def load_parts_csv_two_cols(path: str) -> List[Dict[str, str]]:
    """
    Expected headers (case-insensitive):
      - Name
      - Seq
      - 5' syntax position   (left / 5′ tag)
      - 3' syntax position   (right / 3′ tag)
      - Enzyme               (optional; defaults to BsaI)

    Returns: list of dicts -> {name, seq, left_tag, right_tag, enzyme}
    """
    rows: List[Dict[str, str]] = []
    tag_counts = Counter()
    with open(path, newline="") as fh:
        rdr = csv.DictReader(fh)
        if rdr.fieldnames is None:
            raise ValueError("CSV appears to have no header row.")

        for idx, row in enumerate(rdr, start=1):
            name = _ci_lookup(row, "Name") or f"row_{idx}"
            seq  = _ci_lookup(row, "Seq", "Sequence") or ""
            left = (_ci_lookup(row, "5' syntax position", "5prime", "left", "left_tag") or "").upper()
            right = (_ci_lookup(row, "3' syntax position", "3prime", "right", "right_tag") or "").upper()
            enzyme = (_ci_lookup(row, "Enzyme") or "BsaI").upper()

            # Validate tags
            if not left or not right:
                raise ValueError(f"[{name}] Missing 5′/3′ syntax tags.")
            if left not in VALID_TAGS:
                raise ValueError(f"[{name}] Invalid 5′ tag '{left}'. Allowed: {', '.join(sorted(VALID_TAGS))}")
            if right not in VALID_TAGS:
                raise ValueError(f"[{name}] Invalid 3′ tag '{right}'. Allowed: {', '.join(sorted(VALID_TAGS))}")

            tag_counts[left] += 1
            tag_counts[right] += 1

            # Heads-up if Seq looks like plain DNA (common input mixup)
            if _looks_like_plain_dna(seq):
                print(f"⚠️  {name}: Seq contains only A/C/G/T (looks like DNA). "
                      f"If you intended a protein sequence here, double-check your CSV. "
                      f"(Proceeding as-is; comment out this check if intentional.)")

            # Gentle reminder for non-BsaI rows (so you can route them differently downstream)
            if enzyme != "BSAI":
                print(f"ℹ️  {name}: Enzyme='{enzyme}'. Ensure downstream handling matches this enzyme.")

            rows.append({
                "name": name,
                "seq": seq,
                "left_tag": left,    # 5′ tag (left)
                "right_tag": right,  # 3′ tag (right)
                "enzyme": enzyme,
            })

    print(f"📥 Loaded {len(rows)} row(s) from {path}. Tag usage: {dict(tag_counts)}")
    print("➡️  Convention: 5′ tag applies to the LEFT end; 3′ tag to the RIGHT end.")
    return rows

# --- Upload + load (call AFTER the function is defined) ---
uploaded = files.upload()                 # pick your CSV
csv_path = list(uploaded.keys())[0]       # handles "name (2).csv" etc.
parts = load_parts_csv_two_cols(csv_path)

print(f"\nFirst row preview:\n{parts[0] if parts else 'No rows'}")


KeyboardInterrupt: 

## Step 2: Reverse translate into DNA


In [ ]:
# (Re)init your output container
dna_records = []

# Ensure necessary BioPython imports are available
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from Bio.Data import CodonTable

# Get the bacterial codon table
BACTERIAL_TABLE = CodonTable.unambiguous_dna_by_id[11]

# Create a mapping from amino acid to a list of possible codons
AA_TO_CODONS_LIST: Dict[str, List[str]] = {}
for codon, aa in BACTERIAL_TABLE.forward_table.items():
    AA_TO_CODONS_LIST.setdefault(aa, []).append(codon)

# Iterate over the CSV rows returned by `load_parts_csv_two_cols`
# Each row looks like: {"name", "seq", "left_tag", "right_tag", "enzyme"}
for row in parts:
    name = row["name"]
    protein_seq_str = row["seq"] # The input protein sequence string
    left_tag = row.get("left_tag", "")
    right_tag = row.get("right_tag", "")
    enzyme = row.get("enzyme", "")

    print(f"\nProcessing {name}...")

    # Skip if protein sequence is empty
    if not protein_seq_str:
        print(f"  🚫 Skipping {name} due to empty protein sequence.")
        continue

    initial_dna_seq = "" # Initialize DNA sequence for this record
    translation_successful = True

    try:
        # Perform basic reverse translation manually using the codon table
        dna_sequence_parts = []
        for aa in protein_seq_str:
            # Get possible codons for the amino acid
            codons = AA_TO_CODONS_LIST.get(aa.upper()) # Use upper() for safety

            if not codons:
                # Handle amino acids not found in the codon table (e.g., 'X', 'Z')
                print(f"    ⚠️ Warning: Amino acid '{aa}' not found in bacterial codon table for {name}. Skipping codon.")
                translation_successful = False
                break # Stop translation for this sequence

            # For basic reverse translation, just pick the first codon from the list
            # Note: This is a simplistic approach. BioPython's back_translate might use
            # codon usage frequencies, but this avoids the AttributeError.
            chosen_codon = codons[0]
            dna_sequence_parts.append(chosen_codon)

        if translation_successful and dna_sequence_parts:
            initial_dna_seq = "".join(dna_sequence_parts)
            print(f"  • Basic reverse translation done manually: len={len(initial_dna_seq)} bp")
            # We cannot calculate GC% here without the _gc_pct function being available in this cell's scope
            # print(f"    GC={_gc_pct(initial_dna_seq):.1f}%") # Assumes _gc_pct is defined elsewhere

            # Removed the problematic translation verification step

        elif not translation_successful:
             print(f"  🚫 Skipping {name} due to untranslatable amino acids.")


    except Exception as e:
        print(f"  ❌ Error during manual reverse translation for {name}: {e}")
        initial_dna_seq = "" # Set to empty if translation fails
        continue # Skip this record if translation failed


    # Build a SeqRecord for the initial DNA sequence
    if initial_dna_seq: # Only create record if we have a sequence
        rec = SeqRecord(
            Seq(initial_dna_seq),
            id=name,
            name=name,
            description=f"Initial DNA | {left_tag}|{right_tag}|{enzyme}"
        )
        dna_records.append(rec)
        print(f"  ✅ Added {name} ({len(initial_dna_seq)} bp) to dna_records.")
    else:
        print(f"  🚫 Skipping {name} due to empty sequence after processing or translation failure.")


# Quick inspect of the populated dna_records list
print("\n--- Quick inspect of dna_records ---")
if dna_records:
    for r in dna_records:
        print(f"ID: {r.id}, Length: {len(r.seq)} bp")
else:
    print("dna_records list is empty.")


Processing mVirD2...
  • Basic reverse translation done manually: len=27 bp
  ✅ Added mVirD2 (27 bp) to dna_records.

Processing sfGFP...
  • Basic reverse translation done manually: len=39 bp
  ✅ Added sfGFP (39 bp) to dna_records.

Processing tagRFP...
  • Basic reverse translation done manually: len=36 bp
  ✅ Added tagRFP (36 bp) to dna_records.

--- Quick inspect of dna_records ---
ID: mVirD2, Length: 27 bp
ID: sfGFP, Length: 39 bp
ID: tagRFP, Length: 36 bp


In [ ]:
# # ========================= Codon utilities =========================
# BACTERIAL_TABLE = CodonTable.unambiguous_dna_by_id[11]
# CODON_TO_AA = BACTERIAL_TABLE.forward_table.copy()
# STOP_CODONS = set(BACTERIAL_TABLE.stop_codons)
# AA_TO_CODONS: Dict[str, List[str]] = {}
# for codon, aa in CODON_TO_AA.items():
#     AA_TO_CODONS.setdefault(aa, []).append(codon)

# # Sequences start AFTER ATG; edit codons 1..7 of provided insert
# WOBBLE_WINDOW = (1, 7)
# WOBBLE_FIRST_ALLOWED = {"A", "T"}  # codon 1 wobble
# WOBBLE_OTHERS_ALLOWED = {"A"}      # codons 2..7 wobble
# LOW_G_WINDOW = WOBBLE_WINDOW

# def _hamming(a: str, b: str) -> int:
#     return sum(x != y for x, y in zip(a, b))

# def _ordered_wobble_candidates(aa: str, codon: str, allowed: Set[str]) -> List[str]:
#     if aa not in AA_TO_CODONS:
#         return []
#     first2 = codon[:2]
#     cands = []
#     for wob in sorted(allowed):
#         cand = (first2 + wob).upper()
#         if cand in AA_TO_CODONS[aa]: # Corrected typo here
#             cands.append(cand)
#     others = [c for c in AA_TO_CODONS[aa] if c[-1] in allowed and c not in cands]
#     others.sort(key=lambda c: (_hamming(c, codon), c))
#     cands.extend(others)
#     return cands

# def _ordered_lowG_candidates(aa: str, codon: str, allowed: Optional[Set[str]]) -> List[str]:
#     if aa not in AA_TO_CODONS:
#         return []
#     pool = AA_TO_CODONS[aa]
#     if allowed is not None:
#         pool = [c for c in pool if c[-1] in allowed]
#     pool = [c for c in pool if c.count("G") < codon.count("G")]
#     first2 = codon[:2]
#     pool.sort(key=lambda c: (c.count("G"), 0 if c[:2] == first2 else 1, _hamming(c, codon), c))
#     return pool

# def _passes_constraints_whole(seq_str: str) -> bool:
#     p = DnaOptimizationProblem(sequence=seq_str, constraints=build_constraints())
#     return p.all_constraints_pass()

# def _aa(seq: str) -> str:
#     return str(Seq(seq).translate(table=11))

# def wobble_and_reduce(seq: str) -> Tuple[str, List[Tuple[int, str, str, str]]]:
#     s = str(seq).upper()
#     edits: List[Tuple[int, str, str, str]] = []
#     if len(s) < 3 or len(s) % 3 != 0:
#         return s, edits

#     baseline_aa = _aa(s)  # cache and update after each accepted edit
#     n_codons = len(s) // 3

#     # Pass 1: wobble
#     w_start, w_end = WOBBLE_WINDOW
#     w_end = min(w_end, n_codons)
#     if w_start <= w_end:
#         for idx in range(w_start, w_end + 1):
#             pos = (idx - 1) * 3
#             codon = s[pos:pos+3]
#             if codon in STOP_CODONS:
#                 break
#             aa = CODON_TO_AA.get(codon)
#             if not aa:
#                 continue
#             allowed = WOBBLE_FIRST_ALLOWED if idx == w_start else WOBBLE_OTHERS_ALLOWED
#             if codon[-1] in allowed:
#                 continue
#             for repl in _ordered_wobble_candidates(aa, codon, allowed):
#                 tentative = s[:pos] + repl + s[pos+3:]
#                 if _passes_constraints_whole(tentative) and _aa(tentative) == baseline_aa:
#                     s = tentative
#                     baseline_aa = _aa(s)
#                     edits.append((idx, codon, repl, "wobble"))
#                     break

#     # Pass 2: low-G within window
#     lg_start, lg_end = LOW_G_WINDOW
#     lg_end = min(lg_end, n_codons)
#     if lg_start <= lg_end:
#         for idx in range(lg_start, lg_end + 1):
#             pos = (idx - 1) * 3
#             codon = s[pos:pos+3]
#             if codon in STOP_CODONS:
#                 break
#             aa = CODON_TO_AA.get(codon)
#             if not aa:
#                 continue
#             allowed = WOBBLE_FIRST_ALLOWED if idx == w_start else WOBBLE_OTHERS_ALLOWED
#             for repl in _ordered_lowG_candidates(aa, codon, allowed):
#                 tentative = s[:pos] + repl + s[pos+3:]
#                 if _passes_constraints_whole(tentative) and _aa(tentative) == baseline_aa:
#                     s = tentative
#                     baseline_aa = _aa(s)
#                     edits.append((idx, codon, repl, "lowG"))
#                     break

#     return s, edits

## Enrich AT-rich codons at N terminus




Reference: https://www.science.org/doi/10.1126/science.1241934

 Most synthetic biology studies only investigate the first 10 codons of E. coli, neglecting to provide information on more downstream sequences, even though the “ramp sequence” has been demonstrated to encompass as many as 30–50 codons (Tuller and Zur, 2015). A large-scale comprehensive data analysis of recombinant proteins in E. coli also indicated that the impact of codon usage on protein expression is more significant than the influence of mRNA folding factors(Boël et al., 2016). Recent studies on the ramp sequence suggest that codons 3–5 may act as gates controlling mRNA translation efficiency. The expression of different proteins may not be attributed to alterations in initiation but rather the elongation of codons 4 and 5. The probability of translational arrest during synthesis of the N-terminal pentapeptide may determine the overall efficiency of protein synthesis(Verma et al., 2019; Moreira et al., 2019).

Reference: https://academic.oup.com/nar/article/51/5/2363/7016452?login=false

 Protein production levels and the full coding sequences were determined for 1459 gene variants in Escherichia coli. Using different machine learning approaches, these data were used to reveal correlations between codon usage and protein production. Interestingly, protein production levels can be relatively accurately predicted (Pearson correlation of 0.762) by a Random Forest model that only relies on the sequence information of the first eight codons. In this region, close to the translation initiation site, mRNA secondary structure rather than Codon Adaptation Index (CAI) is the key determinant of protein production. This study clearly demonstrates the key role of codons at the start of the coding sequence. Furthermore, these results imply that commonly used CAI-based codon optimization of the full coding sequence is not a very effective strategy. One should rather focus on optimizing protein production via reducing mRNA secondary structure formation with the first few codons.
....
 Remarkably, we show that only a window covering codons 2–8 is required to accurately predict mRFP production, based on sequence information only. This further strengthens the conclusions from previous studies and demonstrates that other codons later in the CDS in this study are not important to explain protein production. This also underlines that future studies aiming to optimize protein production should focus mostly on the codon usage of the 5′ start of the coding sequence, rather than the current practice of full CDS optimization.

...

The bases 6–25 correspond to codons 2–8. The nucleotides in the third position of these codons that contribute to high expression are mostly A, as well as T in codon 2

 Kudla G., Murray A.W., Tollervey D., Plotkin J.B. Coding-sequence determinants of gene expression in Escherichia coli. Science. 2009; 324:255–258.
 Cambray G., Guimaraes J.C., Arkin A.P. Evaluation of 244,000 synthetic sequences reveals design principles to optimize translation in Escherichia coli. Nat. Biotechnol. 2018; 36:1005–1015.
 Welch M., Govindarajan S., Ness J.E., Villalobos A., Gurney A., Minshull J., Gustafsson C. Design parameters to control synthetic gene expression in Escherichia coli. PLoS One. 2009; 4:e7002.

 Reference: https://www.jbc.org/article/S0021-9258(23)00318-6/fulltext

 The insertion of the DNA sequence encoding SKIK peptide adjacent to the M start codon of a difficult-to-express protein enhances protein production in Escherichia coli. In this report, we reveal that the increased production of the SKIK-tagged protein is not due to codon usage of the SKIK sequence.

 We reported previously that the insertion of codons 5′-TCT AAA ATA AAA-3′ or synonymous codons 5′-TCG AAG ATC AAG-3′ that encode four amino acids, Ser-Lys-Ile-Lys (SKIK), immediately after the Met start codon markedly increased the production level of proteins that were difficult to express in Escherichia coli

 Reference: https://www.biorxiv.org/content/10.1101/2024.03.21.586065v1.abstract

 Protein synthesis efficiency is highly dependent on mRNA coding sequence. Furthermore, there is extensive evidence of a correlation between mRNA stability and protein expression level, though the mechanistic determinants remain unclear. Using yellow fluorescent protein (YFP) as a reporter gene, we herein demonstrate that adenosine (A) abundance in the first six codons is a critical determinant for achieving high protein synthesis in E. coli. Increasing A and/or decreasing guanosine (G) content in this region results in substantial increases in protein expression level both in vivo and in vitro that are correlated with steady-state mRNA concentration in vivo, and this effect is attributable to changes in the stability of the mRNA that are directly coupled to its translation efficiency. Increasing A content promotes mRNA incorporation into the functional 70S ribosomal initiation complex without altering its affinity for the 30S ribosomal subunit. These results support a model in which base composition in the first six codons modulates local mRNA folding energy to control the balance between productive translation initiation versus degradation of mRNAs bound to the 30S ribosomal subunit. Based on these findings, we developed a short N-terminal coding sequence that optimizes translation initiation efficiency for protein production in E. coli.

 Translation boosting sequence is SKIVKI but don't cite the SKIK group!

In [ ]:
# ========================= Codon utilities =========================
BACTERIAL_TABLE = CodonTable.unambiguous_dna_by_id[11]
CODON_TO_AA = BACTERIAL_TABLE.forward_table.copy()
STOP_CODONS = set(BACTERIAL_TABLE.stop_codons)
AA_TO_CODONS: Dict[str, List[str]] = {}
for codon, aa in CODON_TO_AA.items():
    AA_TO_CODONS.setdefault(aa, []).append(codon)

# Sequences start AFTER ATG; edit codons 1..7 of provided insert
WOBBLE_WINDOW = (1, 7)
WOBBLE_FIRST_ALLOWED = {"A", "T"}  # codon 1 wobble
WOBBLE_OTHERS_ALLOWED = {"A"}      # codons 2..7 wobble
LOW_G_WINDOW = WOBBLE_WINDOW

def _hamming(a: str, b: str) -> int:
    return sum(x != y for x, y in zip(a, b))

def _ordered_wobble_candidates(aa: str, codon: str, allowed: Set[str]) -> List[str]:
    if aa not in AA_TO_CODONS:
        return []
    first2 = codon[:2]
    cands = []
    for wob in sorted(allowed):
        cand = (first2 + wob).upper()
        if cand in AA_TO_CODONS[aa]:
            cands.append(cand)
    others = [c for c in AA_TO_CODONS[aa] if c[-1] in allowed and c not in cands]
    others.sort(key=lambda c: (_hamming(c, codon), c))
    cands.extend(others)
    return cands

def _ordered_lowG_candidates(aa: str, codon: str, allowed: Optional[Set[str]]) -> List[str]:
    if aa not in AA_TO_CODONS:
        return []
    pool = AA_TO_CODONS[aa]
    if allowed is not None:
        pool = [c for c in pool if c[-1] in allowed]
    pool = [c for c in pool if c.count("G") < codon.count("G")]
    first2 = codon[:2]
    pool.sort(key=lambda c: (c.count("G"), 0 if c[:2] == first2 else 1, _hamming(c, codon), c))
    return pool

def _passes_constraints_whole(seq_str: str) -> bool:
    p = DnaOptimizationProblem(sequence=seq_str, constraints=build_constraints())
    return p.all_constraints_pass()

def _aa(seq: str) -> str:
    return str(Seq(seq).translate(table=11))

def wobble_and_reduce(seq: str) -> Tuple[str, List[Tuple[int, str, str, str]]]:
    s = str(seq).upper()
    edits: List[Tuple[int, str, str, str]] = []
    if len(s) < 3 or len(s) % 3 != 0:
        return s, edits

    baseline_aa = _aa(s)  # cache AA
    n_codons = len(s) // 3

    # Pass 1: wobble
    w_start, w_end = WOBBLE_WINDOW
    w_end = min(w_end, n_codons)
    if w_start <= w_end:
        for idx in range(w_start, w_end + 1):
            pos = (idx - 1) * 3
            codon = s[pos:pos+3]
            if codon in STOP_CODONS:
                break
            aa = CODON_TO_AA.get(codon)
            if not aa:
                continue
            allowed = WOBBLE_FIRST_ALLOWED if idx == w_start else WOBBLE_OTHERS_ALLOWED
            if codon[-1] in allowed:
                continue
            for repl in _ordered_wobble_candidates(aa, codon, allowed):
                tentative = s[:pos] + repl + s[pos+3:]
                if _passes_constraints_whole(tentative) and _aa(tentative) == baseline_aa:
                    s = tentative
                    baseline_aa = _aa(s)
                    edits.append((idx, codon, repl, "wobble"))
                    break

    # Pass 2: low-G within window
    lg_start, lg_end = LOW_G_WINDOW
    lg_end = min(lg_end, n_codons)
    if lg_start <= lg_end:
        for idx in range(lg_start, lg_end + 1):
            pos = (idx - 1) * 3
            codon = s[pos:pos+3]
            if codon in STOP_CODONS:
                break
            aa = CODON_TO_AA.get(codon)
            if not aa:
                continue
            allowed = WOBBLE_FIRST_ALLOWED if idx == w_start else WOBBLE_OTHERS_ALLOWED
            for repl in _ordered_lowG_candidates(aa, codon, allowed):
                tentative = s[:pos] + repl + s[pos+3:]
                if _passes_constraints_whole(tentative) and _aa(tentative) == baseline_aa:
                    s = tentative
                    baseline_aa = _aa(s)
                    edits.append((idx, codon, repl, "lowG"))
                    break

    return s, edits




## DNA Chisel check after codon optimisation and wobble inserts

In [ ]:
# ========================= Insert-only DNA Chisel check =========================
def build_insert_only_constraints(insert_start: int, insert_end: int):
    loc = (insert_start, insert_end)
    return [
        AvoidPattern("BsaI_site", strand='both', location=loc),
        AvoidPattern("SapI_site", strand='both', location=loc),
        AvoidPattern("BbsI_site", strand='both', location=loc),
        AvoidPattern("BsmBI_site", strand='both', location=loc),
        AvoidPattern("EcoRI_site", strand='both', location=loc),
        AvoidPattern("NotI_site", strand='both', location=loc),
        AvoidPattern("XbaI_site", strand='both', location=loc),
        AvoidPattern("SpeI_site", strand='both', location=loc),
        AvoidPattern("PstI_site", strand='both', location=loc),
        AvoidPattern("BtgZI_site", strand='both', location=loc),
        AvoidPattern("AarI_site", strand='both', location=loc),
        AvoidPattern("NheI_site", strand='both', location=loc),
        AvoidPattern("XhoI_site", strand='both', location=loc),
        AvoidPattern("BamHI_site", strand='both', location=loc),
        AvoidPattern("BglII_site", strand='both', location=loc),
        AvoidPattern("NruI_site", strand='both', location=loc),
        AvoidPattern(HomopolymerPattern("A", 6), location=loc),
        AvoidPattern(HomopolymerPattern("T", 6), location=loc),
        AvoidPattern(HomopolymerPattern("C", 6), location=loc),
        AvoidPattern(HomopolymerPattern("G", 6), location=loc),
        EnforceGCContent(mini=0.4, maxi=0.6, window=100, location=loc),
        EnforceTranslation(genetic_table='Bacterial', location=loc),
    ]

def check_insert_region(final_construct: str, insert_start: int, insert_end: int):
    p = DnaOptimizationProblem(
        sequence=final_construct,
        constraints=build_insert_only_constraints(insert_start, insert_end),
        objectives=[]
    )
    return p.all_constraints_pass(), p.constraints_text_summary()



## Add flanking regions for Type IIS Assembly

In [ ]:
# ========================= Builders (simplified) =========================
def make_popenv3_only(
    insert_seq: str,
    *,
    flank5: str = DEFAULT_FLANK5, flank3: str = DEFAULT_FLANK3,
    nn_bbs: str = DEFAULT_BBSI_NN,
):
    insert = str(insert_seq).upper()
    left  = (flank5 + BBSI_SITE_FW + nn_bbs + BBSI_LEFT_OVERHANG).upper()
    right = (BBSI_RIGHT_OVERHANG + nn_bbs + BBSI_SITE_RV + flank3).upper()
    construct = left + insert + right
    meta = {
        "insert_start": len(left), "insert_end": len(left) + len(insert),
        "flank5": flank5.upper(), "flank3": flank3.upper(), "nn_bbs": nn_bbs.upper(),
    }
    return construct, meta

def make_cd_part_with_bsaI(
    insert_seq: str,
    *,
    left_bsa_letter: str = "C", right_bsa_letter: str = "D",
    flank5: str = DEFAULT_FLANK5, flank3: str = DEFAULT_FLANK3,
    nn_bbs: str = DEFAULT_BBSI_NN,
    spacer_between_sites: str = DEFAULT_SPACER_BETWEEN_SITES,
    spacer_right: str = DEFAULT_SPACER_RIGHT,
    n_bsa: str = DEFAULT_BSAI_N,
):
    """
    Reclone CD part (top strand):
      flank5 BBSI_SITE_FW nn_bbs CGCT spacer_between_sites
      BSAI_SITE_FW n_bsa [LEFT5_PREFIX] LEFT_BSA [LEFT5_SUFFIX]  [INSERT]
      [RIGHT3_PREFIX] RIGHT_BSA [RIGHT3_SUFFIX] n_bsa BSAI_SITE_RV spacer_right
      GGAG nn_bbs BBSI_SITE_RV flank3
    """
    # Resolve position-aware extras
    Lpre, Lbsa, Lsuf = resolve_bsa_end(left_bsa_letter,  side="5")
    Rpre, Rbsa, Rsuf = resolve_bsa_end(right_bsa_letter, side="3")

    insert = str(insert_seq).upper()

    # LEFT segment up to the insert start (includes any 5′ prefix/suffix around the left overhang)
    left = (
        flank5 + BBSI_SITE_FW + nn_bbs + BBSI_LEFT_OVERHANG +
        spacer_between_sites + BSAI_SITE_FW + n_bsa +
        Lpre + Lbsa + Lsuf
    ).upper()

    # RIGHT segment after the insert end (may include a 3′ prefix BEFORE the right overhang and a suffix AFTER it)
    right = (
        Rpre + Rbsa + Rsuf + n_bsa + BSAI_SITE_RV +
        spacer_right + BBSI_RIGHT_OVERHANG + nn_bbs + BBSI_SITE_RV + flank3
    ).upper()

    construct = left + insert + right
    meta = {
        "insert_start": len(left), "insert_end": len(left) + len(insert),
        "flank5": flank5.upper(), "flank3": flank3.upper(), "nn_bbs": nn_bbs.upper(),
        "spacer_between_sites": spacer_between_sites.upper(), "spacer_right": spacer_right.upper(),
        "n_bsa": n_bsa.upper(),
        "LEFT_BSA_PRE": Lpre, "LEFT_BSA": Lbsa, "LEFT_BSA_SUF": Lsuf,
        "RIGHT_BSA_PRE": Rpre, "RIGHT_BSA": Rbsa, "RIGHT_BSA_SUF": Rsuf,
    }
    return construct, meta

def build_final_part(insert_seq: str, mode="cd", **kwargs):
    if mode == "cd":
        return make_cd_part_with_bsaI(insert_seq, **kwargs)
    elif mode == "popenv3":
        return make_popenv3_only(insert_seq, **kwargs)
    else:
        raise ValueError("mode must be 'cd' or 'popenv3'")



## Guardrail Checks

In [ ]:
def _assert_slice(seqU: str, start: int, token: str, label: str) -> int:
    if not token:
        return start  # allow zero-length extras
    end = start + len(token)
    if seqU[start:end] != token:
        got = seqU[start:end]
        raise ValueError(f"Guardrail fail at {label}: expected '{token}' at {start}:{end}, got '{got}'")
    return end

def assert_cd_layout(final_seq: str, meta: dict) -> bool:
    s = final_seq.upper(); i = 0
    i = _assert_slice(s, i, meta["flank5"], "flank5")
    i = _assert_slice(s, i, BBSI_SITE_FW, "BbsI_fw_site")
    i = _assert_slice(s, i, meta["nn_bbs"], "BbsI_NN")
    i = _assert_slice(s, i, BBSI_LEFT_OVERHANG, "Left_Bbs_overhang")
    i = _assert_slice(s, i, meta["spacer_between_sites"], "spacer_between_sites")
    i = _assert_slice(s, i, BSAI_SITE_FW, "BsaI_fw_site")
    i = _assert_slice(s, i, meta["n_bsa"], "BsaI_N")

    # 5′ extras before/after left BsaI overhang
    i = _assert_slice(s, i, meta["LEFT_BSA_PRE"], "Left_Bsa_prefix")
    i = _assert_slice(s, i, meta["LEFT_BSA"], "Left_Bsa_overhang")
    i = _assert_slice(s, i, meta["LEFT_BSA_SUF"], "Left_Bsa_suffix")

    # Insert starts here
    if i != meta["insert_start"]:
        raise ValueError(f"Guardrail fail: insert_start mismatch. Expected {meta['insert_start']}, got {i}")

    # Skip the insert region
    i = meta["insert_end"]

    # 3′ extras: optional prefix BEFORE right overhang, suffix AFTER
    i = _assert_slice(s, i, meta["RIGHT_BSA_PRE"], "Right_Bsa_prefix")
    i = _assert_slice(s, i, meta["RIGHT_BSA"], "Right_Bsa_overhang")
    i = _assert_slice(s, i, meta["RIGHT_BSA_SUF"], "Right_Bsa_suffix")

    i = _assert_slice(s, i, meta["n_bsa"], "BsaI_N_2")
    i = _assert_slice(s, i, BSAI_SITE_RV, "BsaI_rev_site")
    i = _assert_slice(s, i, meta["spacer_right"], "spacer_right")
    i = _assert_slice(s, i, BBSI_RIGHT_OVERHANG, "Right_Bbs_overhang")
    i = _assert_slice(s, i, meta["nn_bbs"], "BbsI_NN_2")
    i = _assert_slice(s, i, BBSI_SITE_RV, "BbsI_rev_site_2")
    i = _assert_slice(s, i, meta["flank3"], "flank3")

    if i != len(s):
        raise ValueError("Guardrail fail: trailing sequence after expected end.")
    return True



## Build and print outputs

In [ ]:
# ========================= Verbose per-record processing + outputs =========================
# CONFIG you may tweak
BUILD_MODE = "cd"          # "cd" or "popenv3"
# Use tags from CSV if available, otherwise fall back to these defaults
DEFAULT_LEFT_BSA_LETTER  = "C"
DEFAULT_RIGHT_BSA_LETTER = "D"

# Optional: linkers/flanks (keep defaults unless you need to change)
LINKER_BETWEEN_SITES = DEFAULT_SPACER_BETWEEN_SITES
LINKER_RIGHT         = DEFAULT_SPACER_RIGHT
BBSI_NN              = DEFAULT_BBSI_NN
BSAI_N               = DEFAULT_BSAI_N
FLANK5               = DEFAULT_FLANK5
FLANK3               = DEFAULT_FLANK3

# Ensure we have places to put results
optimized_sequences: List[SeqRecord] = [] # Enriched inserts (no adapters)
final_sequences: List[str] = [] # Raw sequence strings of enriched inserts
final_records: List[SeqRecord] = [] # SeqRecords of enriched inserts (redundant with optimized_sequences?)
synth_records: List[SeqRecord] = [] # Synthesis-ready parts (with adapters)

print(f"\n=== Building parts | mode={BUILD_MODE}"
      + (f" | Default BsaI {DEFAULT_LEFT_BSA_LETTER}->{DEFAULT_RIGHT_BSA_LETTER}" if BUILD_MODE=='cd' else "")
      + f" | n={len(dna_records) if 'dna_records' in locals() else 0} ===") # Check if dna_records is defined

# Ensure dna_records is defined and not empty before proceeding
if 'dna_records' not in locals() or not dna_records:
    print("🚫 dna_records list is empty or not defined. Please run the previous step (reverse translation) first.")
else:
    for i, rec in enumerate(dna_records):
        rec_name = getattr(rec, "name", None) or getattr(rec, "id", None) or f"insert_{i+1}"
        initial_dna_seq = str(rec.seq) # Get the initial DNA from the dna_records list

        print(f"\n[{i+1}/{len(dna_records)}] Processing {rec_name}")
        print(f"  • Starting with initial DNA: len={len(initial_dna_seq)} bp | GC={_gc_pct(initial_dna_seq):.1f}% | G={initial_dna_seq.count('G')}")

        # 1) Optimize with DNA Chisel (starting from the initial DNA)
        # Note: If you started with protein in the previous step, this DNA is
        # already a basic reverse translation. This step optimizes it further.
        problem = DnaOptimizationProblem(
            sequence=initial_dna_seq, # Start with the initial DNA sequence
            constraints=build_constraints(), # Assumes build_constraints is defined
            objectives=[CodonOptimize(species='e_coli')] # Optimize codons
        )
        print("  • Starting DNA Chisel optimization...")
        try:
            problem.resolve_constraints()
            problem.optimize()
            final_optimized_dna = problem.sequence
            print(f"  • DNA Chisel optimized: len={len(final_optimized_dna)} bp | GC={_gc_pct(final_optimized_dna):.1f}% | G={final_optimized_dna.count('G')}")
        except Exception as e:
            print(f"  ❌ Error during DNA Chisel optimization for {rec_name}: {e}")
            final_optimized_dna = initial_dna_seq # Revert to initial DNA if optimization fails
            print("    Using initial DNA sequence for subsequent steps.")


        # 2) Wobble/lowG (first 7 codons only) - applied to the optimized DNA
        enriched_seq, edits = wobble_and_reduce(final_optimized_dna) # Assumes wobble_and_reduce is defined
        print(f"  • wobble/lowG edits: n={len(edits)} | ΔG={enriched_seq.count('G') - final_optimized_dna.count('G')}"
              f" | GC { _gc_pct(final_optimized_dna):.1f}% → { _gc_pct(enriched_seq):.1f}%")
        print(f"    edits: {_short_edits(edits)}")

        # AA safety check (compare translation of optimized DNA vs enriched DNA)
        try:
            aa_ok = (_aa(final_optimized_dna).rstrip("*") == _aa(enriched_seq).rstrip("*")) # Assumes _aa is defined
            print(f"  • AA unchanged after edits? {'YES' if aa_ok else 'NO (unexpected!)'}")
            if not aa_ok:
                print("    ⚠️ Reverting to DNA Chisel output to preserve translation.")
                enriched_seq = final_optimized_dna # Revert to DNA from CodonOptimize if AA changes
                edits = []
        except Exception as e:
             print(f"  ⚠️ Warning: Could not verify AA unchanged after edits: {e}. Proceeding with enriched sequence.")
             aa_ok = False # Cannot confirm, mark as potentially not OK

        # Whole-insert constraint check (on the enriched sequence)
        check_problem_enriched = DnaOptimizationProblem(
            sequence=enriched_seq,
            constraints=build_constraints() # Assumes build_constraints is defined
        )
        constraints_ok_enriched = check_problem_enriched.all_constraints_pass()
        if constraints_ok_enriched:
            print(f"  • Constraints after edits: ✅ PASS")
        else:
            print(f"  • Constraints after edits: ❌ FAIL — details below")
            print(check_problem_enriched.constraints_text_summary())


        # Save enriched insert (no adapters yet)
        optimized_sequences.append(
            SeqRecord(Seq(enriched_seq), id=rec_name,
                      description=f"Optimized + wobble/lowG | edits={len(edits)} | "
                                  f"AA={'OK' if aa_ok else 'MISMATCH'} | constraints={'OK' if constraints_ok_enriched else 'FAIL'}")
        )
        final_sequences.append(enriched_seq)
        # Add the enriched sequence as a record for final_records (used by problem.to_record potentially, but simpler to just store the seq)
        # If you need the full DnaOptimizationProblem results, you'd save 'problem' or its summary here.
        final_records.append(SeqRecord(Seq(enriched_seq), id=rec_name + "_enriched", description="Enriched sequence"))


        # 3) Build adapters + guardrails + insert-only check
        # Use tags from the original row in 'parts' if available, fallback to defaults
        # Need to find the corresponding original row in 'parts'
        original_row = next((item for item in parts if item["name"] == rec_name), None)
        if original_row:
             current_left_tag = original_row.get("left_tag", DEFAULT_LEFT_BSA_LETTER)
             current_right_tag = original_row.get("right_tag", DEFAULT_RIGHT_BSA_LETTER)
             print(f"  • Using tags from CSV: {current_left_tag}-{current_right_tag}")
        else:
             current_left_tag = DEFAULT_LEFT_BSA_LETTER
             current_right_tag = DEFAULT_RIGHT_BSA_LETTER
             print(f"  • Original row not found for {rec_name}. Using default tags: {current_left_tag}-{current_right_tag}")


        if BUILD_MODE == "cd":
            try:
                final_construct, meta = build_final_part(
                    enriched_seq, mode="cd",
                    left_bsa_letter=current_left_tag, right_bsa_letter=current_right_tag,
                    flank5=FLANK5, flank3=FLANK3,
                    nn_bbs=BBSI_NN, n_bsa=BSAI_N,
                    spacer_between_sites=LINKER_BETWEEN_SITES, spacer_right=LINKER_RIGHT,
                )
                print("  • Built CD layout with adapters.")
                # Guardrails: exact motif placement
                try:
                    assert_cd_layout(final_construct, meta) # Assumes assert_cd_layout is defined
                    print("    Guardrails: ✅ CD layout OK")
                    guardrail_status = "PASS"
                except Exception as e:
                    print(f"    Guardrails: ❌ CD layout FAILED — {e}")
                    guardrail_status = f"FAIL: {e}"

            except Exception as e:
                 print(f"  ❌ Error building CD part for {rec_name}: {e}")
                 final_construct = ""
                 meta = {}
                 guardrail_status = f"BUILD ERROR: {e}"

        else: # BUILD_MODE == "popenv3"
            try:
                final_construct, meta = build_final_part(
                    enriched_seq, mode="popenv3",
                    flank5=FLANK5, flank3=FLANK3,
                    nn_bbs=BBSI_NN
                )
                print("  • Built pOpenv3 layout with adapters.")
                # Guardrails: exact motif placement
                try:
                    assert_popenv3_layout(final_construct, meta) # Assumes assert_popenv3_layout is defined
                    print("    Guardrails: ✅ pOpenv3 layout OK")
                    guardrail_status = "PASS"
                except Exception as e:
                    print(f"    Guardrails: ❌ pOpenv3 layout FAILED — {e}")
                    guardrail_status = f"FAIL: {e}"
            except Exception as e:
                 print(f"  ❌ Error building pOpenv3 part for {rec_name}: {e}")
                 final_construct = ""
                 meta = {}
                 guardrail_status = f"BUILD ERROR: {e}"


        # Insert-only DNA Chisel check (we ignore adapters on purpose)
        if final_construct and meta: # Only check if construct was built
            try:
                ok_insert, summary = check_insert_region(final_construct, meta["insert_start"], meta["insert_end"]) # Assumes check_insert_region is defined
                print(f"  • Insert-only constraints after adapters: {'✅ PASS' if ok_insert else '❌ FAIL'}")
                insert_check_status = 'OK' if ok_insert else 'FAIL'
                if not ok_insert:
                    print("    Details:")
                    print(summary)
            except Exception as e:
                 print(f"  ❌ Error during insert-only check for {rec_name}: {e}")
                 insert_check_status = f"CHECK ERROR: {e}"
        else:
            print("  • Skipping insert-only constraints check due to build errors.")
            insert_check_status = "SKIPPED"


        # Save adapterized construct
        if final_construct: # Only add if construct was built
            mode_tag = "CD" if BUILD_MODE == "cd" else "pOpenv3"
            extra_tag = (f"({meta.get('LEFT_BSA', '?')}-{meta.get('RIGHT_BSA', '?')})" if BUILD_MODE == "cd" else "")
            synth_records.append(
                SeqRecord(
                    Seq(final_construct),
                    id=f"{rec_name}|{mode_tag}{extra_tag}",
                    description=f"insert={meta.get('insert_start', '?')}..{meta.get('insert_end', '?')} | insert-check={insert_check_status} | guardrails={guardrail_status}"
                )
            )
        else:
             print(f"  🚫 Skipping synthesis part record for {rec_name} due to build errors.")


# ======================= AFTER THE LOOP: write files =======================
if optimized_sequences: # Only write if there were sequences processed
    out1 = "optimized_sequences_enriched_checked.fasta"
    out2 = "synthesis_parts.fasta"
    try:
        with open(out1, "w") as fh1:
            SeqIO.write(optimized_sequences, fh1, "fasta")
        print(f"\n✅ Done writing FASTA outputs.")
        print(f"   • Enriched inserts (no adapters): {out1}  (n={len(optimized_sequences)})")
    except Exception as e:
        print(f"\n❌ Error writing {out1}: {e}")

    try:
        with open(out2, "w") as fh2:
            SeqIO.write(synth_records, fh2, "fasta")
        print(f"   • Synthesis-ready parts ({mode_tag}): {out2}  (n={len(synth_records)})")
    except Exception as e:
        print(f"❌ Error writing {out2}: {e}")

else:
    print("\n🚫 No sequences were processed successfully to generate outputs.")


=== Building parts | mode=cd | Default BsaI C->D | n=3 ===

[1/3] Processing mVirD2
  • Starting with initial DNA: len=27 bp | GC=40.7% | G=4
  • Starting DNA Chisel optimization...


location:   0%|          | 0/6 [00:00<?, ?it/s, now=0-3] 


  • DNA Chisel optimized: len=27 bp | GC=63.0% | G=8
  • wobble/lowG edits: n=6 | ΔG=-3 | GC 63.0% → 44.4%
    edits: 1:AGC>AGT(wobble), 3:CAG>CAA(wobble), 4:CCG>CCA(wobble), 5:ACC>ACA(wobble), 6:CGC>CGA(wobble), 1:AGT>TCT(lowG)
  • AA unchanged after edits? YES
  • Constraints after edits: ✅ PASS
  • Using tags from CSV: C-D
  • Built CD layout with adapters.
    Guardrails: ✅ CD layout OK
  • Insert-only constraints after adapters: ✅ PASS

[2/3] Processing sfGFP
  • Starting with initial DNA: len=39 bp | GC=33.3% | G=9
  • Starting DNA Chisel optimization...


location:   0%|          | 0/8 [00:00<?, ?it/s, now=3-6] 


  • DNA Chisel optimized: len=39 bp | GC=56.4% | G=14
  • wobble/lowG edits: n=3 | ΔG=-2 | GC 56.4% → 48.7%
    edits: 2:AGC>TCA(wobble), 4:GGC>GGA(wobble), 7:CTG>CTA(wobble)
  • AA unchanged after edits? YES
  • Constraints after edits: ✅ PASS
  • Using tags from CSV: B-C
  • Built CD layout with adapters.
    Guardrails: ✅ CD layout OK
  • Insert-only constraints after adapters: ✅ PASS

[3/3] Processing tagRFP
  • Starting with initial DNA: len=36 bp | GC=41.7% | G=6
  • Starting DNA Chisel optimization...


location:   0%|          | 0/6 [00:00<?, ?it/s, now=0-3] 
                                                                                  

  • DNA Chisel optimized: len=36 bp | GC=58.3% | G=8
  • wobble/lowG edits: n=3 | ΔG=-1 | GC 58.3% → 50.0%
    edits: 1:ACC>ACA(wobble), 2:GGC>GGA(wobble), 3:AGC>TCA(wobble)
  • AA unchanged after edits? YES
  • Constraints after edits: ✅ PASS
  • Using tags from CSV: N1-N5
  • Built CD layout with adapters.
    Guardrails: ✅ CD layout OK
  • Insert-only constraints after adapters: ✅ PASS

✅ Done writing FASTA outputs.
   • Enriched inserts (no adapters): optimized_sequences_enriched_checked.fasta  (n=3)
   • Synthesis-ready parts (CD): synthesis_parts.fasta  (n=3)


In [ ]:
# ======================= Generate updated CSV =======================
import pandas as pd

# Ensure 'parts' (from the input CSV) and 'synth_records' (from the processing cell) are defined

# Create a list of dictionaries to hold the combined data
combined_data = []
# Create a dictionary mapping synth_record ID (part name part) to sequence for easy lookup
# Split the ID to get the base part name before the "|"
synth_seq_map = {rec.id.split('|')[0]: str(rec.seq) for rec in synth_records}

# Assuming 'parts' list is available from the CSV loading step
# If 'parts' is not defined here, you might need to add a check or reload it.
try:
    parts
except NameError:
    print("Error: 'parts' data not found. Please run the CSV upload cell first.")
    parts = [] # Initialize as empty to avoid further errors

if parts:
    for row in parts:
        row_name = row.get("name")
        if row_name in synth_seq_map:
            # Create a new dictionary for the row, copying original data
            new_row = row.copy()
            # Add the synthesis sequence
            new_row["synthesis_seq"] = synth_seq_map[row_name]
            combined_data.append(new_row)
        else:
             # If a synth record wasn't generated (e.g., due to errors), still include the original row
             # and indicate no synthesis sequence was generated.
             new_row = row.copy()
             new_row["synthesis_seq"] = "" # Or a placeholder like "Error/Skipped"
             combined_data.append(new_row)
             print(f"⚠️ Warning: No synthesis sequence generated for {row_name}. Excluding from CSV or marking as empty.")


    # Create a pandas DataFrame from the combined data
    if combined_data:
        try:
            df_output = pd.DataFrame(combined_data)
            output_csv_path = "processed_parts_with_synthesis_seq.csv"
            df_output.to_csv(output_csv_path, index=False)
            print(f"\n✅ Generated updated CSV: {output_csv_path}")
        except Exception as e:
            print(f"\n❌ Error generating or writing CSV: {e}")
    else:
        print("\n🚫 No combined data to write to CSV.")
else:
    print("\n🚫 'parts' list is empty. Cannot generate updated CSV.")


✅ Generated updated CSV: processed_parts_with_synthesis_seq.csv
